# Исследование надежности заемщиков


## 1. Описание проекта и цель исследования

**Цель исследования:** Заказчик — кредитный отдел банка. Необходимо проанализировать входные данные от банка — статистику о платёжеспособности клиентов для ответа на вопросы:

* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

Результаты исследования будут учтены при построении модели кредитного скоринга.

**План работы**

* Обзор данных для проверки на ошибки
* Предобработка данных для устранения найденных проблем
* Анализ данных для ответа на поставленные вопросы
* Вывод

## 2. Предобработка данных

In [1]:
#загрузим библиотеки и данные
import pandas as pd

data = pd.read_csv('C:/Users/Yulia/Desktop/Yandex Practicum/Проекты/2- Предобработка данных (банк)/data.csv')

In [2]:
#ознакомимся с данными
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
#изучим информацию о данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
#выведем количество пропусков
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
#заполним пропуски в столбце total_income медианным значением по каждому типу из столбца income_type
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

In [6]:
#обработаем аномалии в столбце days_employed
data['days_employed'] = data['days_employed'].abs()

In [30]:
#для каждого типа занятости выведем медианное значение трудового стажа days_employed в днях
data.groupby('income_type')['days_employed'].agg('median').round(2)

income_type
безработный        366413.65
в декрете            3296.76
госслужащий          2689.14
компаньон            1555.95
пенсионер          365286.62
предприниматель       520.85
сотрудник            1572.33
студент               578.75
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Так как для дальнейшего анализа эти данные нам не понадобяться, было принято решение оставить их как есть. 

In [8]:
#посмотрим на уникальные значения столбца children
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [9]:
#обработаем аномальные значения 
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
#убедимся, что аномалии удалены
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

In [11]:
#заполним пропуски в столбце days_employed медианными значениями по каждому типу занятости income_type
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
#проверим, что все пропуски заполнены
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [13]:
#заменим вещественный тип данных в столбце total_income на целочисленный
data['total_income'] = data['total_income'].astype(int)

In [14]:
#обработаем неявные дубликаты в столбце education, приведием их к нижнему регистру
data['education'] = data['education'].str.lower()

In [15]:
#найдем количество строк-дубликатов в данных
data.duplicated().sum()

71

In [16]:
#удалим дубликаты
data = data.drop_duplicates()

Создадим столбец total_income_category и категорезируем данные по уровню дохода следующим образом:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [17]:
#категоризируем данные
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
#применим функцию
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
#выведем перечень уникальных целей взятия кредита из столбца purpose
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [20]:
#создадим функцию 
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
#категоризируем данные
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## 3. Исследовательский анализ

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
#проверяем количество клиентов в каждой категории по количеству детей
data['children'].value_counts()

0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

In [23]:
#посмотрим долю каждой категории в процентах
data_children = data.groupby('children')['children'].count() #считаем количество клиентов в кажой категории
data_children_share = (data_children / data['children'].count()) * 100 #делим на общее количество клиентов для расчета процентного соотношения
data_children_share

children
0    66.058788
1    22.539965
2     9.619802
3     1.547044
4     0.192209
5     0.042192
Name: children, dtype: float64

**Промежуточный вывод**: доля клиентов с 2-мя детьми и более в сумме составляет около 11%, поэтому для дальнейшего анализа имеет смысл клиентов с 2-мя детьми и более объединить в одну категорию.

In [24]:
#создадим функцию для категоризации все клиентов по трем категорям - Нет детей, Один ребенок, Несколько детей
def children_category(row):
    try:
        if row == 0:
            return 'Нет детей'
        elif row == 1:
            return 'Один ребенок'
        return 'Несколько детей'
    except:
        return 'Нет категории'

In [25]:
#применим функцию к столбцу Children и создадим новый столбец Children_category с категорией
data['children_category'] = data['children'].apply(children_category)
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,children_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,Один ребенок
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,Один ребенок
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,Нет детей
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,Несколько детей
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,Нет детей


In [26]:
#рассчитаем долю клиентов с задолженностью по возврату кредита по категорям
data_children = data.groupby('children_category').agg({'debt' : ['sum', 'count',lambda x: '{:.2%}'.format(x.mean())]})
#переименуем столбцы
data_children = data_children.rename(columns = {'sum': 'number_of_clients_with_debt','count': 'total_number_of_clients','<lambda_0>': 'percentage(%)'}) 
#отсортируем таблицу по возрастанию значений в столбце percentage(%)
data_children.sort_values(by=[('debt','percentage(%)')])

debt                          \
                  number_of_clients_with_debt total_number_of_clients   
children_category                                                       
Нет детей                                1063                   14091   
Один ребенок                              444                    4808   
Несколько детей                           225                    2432   

                                 
                  percentage(%)  
children_category                
Нет детей                 7.54%  
Один ребенок              9.23%  
Несколько детей           9.25%

**Вывод:** 

По итогам таблицы, доля клиентов с задолженностью по возврату кредита составляет:
* Клиенты без детей - 7,54%
* Клиенты с одним ребенком - 9,23%
* Клиенты с несколькими детьми - 9,25% 

Как видно из таблицы выше самая низкая доля задолженности по возврату кредита у клиентов без детей. У клиентов с одним ребеноком или несколькими детьми доля задолженности почти одинаковая и на 1.7% выше, чем у клиетов без детей.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [31]:
#построим сводную таблицу по семейному статусу и рассчитаем долю клиентов с задолженностью по возврату кредита
data_pivot_family = data.pivot_table(index=['family_status_id', 'family_status'], values='debt', aggfunc=['sum', 'count', lambda x: '{:.2%}'.format(x.mean())])
#переименуем столбцы
data_pivot_family = data_pivot_family.rename(columns= {'sum': 'number_of_clients_with_debt', 'count': 'total_number_of_clients', '<lambda>': 'percentage(%)', 'debt': ''})
#отсортируем таблицу по возрастанию значений в столбце percentage(%)
data_pivot_family.sort_values(by='percentage(%)')

,,number_of_clients_with_debt,total_number_of_clients,percentage(%)
,,,,
family_status_id,family_status,,,
2,вдовец / вдова,63,951,6.62%
3,в разводе,84,1189,7.06%
0,женат / замужем,927,12261,7.56%
1,гражданский брак,385,4134,9.31%
4,Не женат / не замужем,273,2796,9.76%


**Вывод:** 

По итогам таблицы доля клиентов с задолженностью по возврату кредита составляет:
* вдовец / вдова - 6,62%
* в разводе - 7,06%
* женат / замужем - 7,56%
* гражданский брак - 9,31%
* Не женат / не замужем - 9,76% 

Как видно из расчетов выше, самый низкий процент наличия задолженности у клиентов в категории "вдовец / вдова" и у клиентов в категории "в разводе". Однако, необходимо отметить, что данные клиенты представляют только ~10% от общего количества клиентов. Самая широко представленная категория клиентов - это клиенты состоящие в официальном браке. Процент наличия задолженности таких клиентов ниже процента наличия задолженности у клиентов состоящих в гражданском браке или холостых клиентов. Это позволяет сделать вывод о том, что клиенты состоящие в официальном браке более надежные клиенты для банка.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [32]:
#построим сводную таблицу по семейному статусу и рассчитаем долю клиентов с задолженностью по возврату кредита
data_pivot_income = data.pivot_table(index=['total_income_category'], values='debt', aggfunc= ['sum', 'count', lambda x: '{:.2%}'.format(x.mean())])
#переименуем столбцы
data_pivot_income = data_pivot_income.rename(columns= {'sum':'number_of_clients_with_debt', 'count':'total_number_of_clients', '<lambda>': 'percentage(%)', 'debt':''})
#добавим новый столбец со диапазоном дохода для каждой категории
new_col = ['>1.000.000', '200.000-1.000.000', '50.000-200.000','30.000-50.000', '<30.000']
data_pivot_income.insert(loc=0, column='range_of_income', value=new_col)
data_pivot_income

,range_of_income,number_of_clients_with_debt,total_number_of_clients,percentage(%)
,,,,
total_income_category,,,,
A,>1.000.000,2,25,8.00%
B,200.000-1.000.000,354,5014,7.06%
C,50.000-200.000,1353,15921,8.50%
D,30.000-50.000,21,349,6.02%
E,<30.000,2,22,9.09%


**Вывод:** 

По итогам таблицы доля клиентов с задолженностью по возврату кредита составляет:
* Категория D (30.000-50.000) - 6,02%
* Категория В (200.000-1.000.000) - 7,06%
* Категория A (>1.000.000) - 8,00%
* Категория С (50.000-200.000) - 8,50%
* Категория Е (<30.000) - 9,09% 

Как видно из расчетов, клиенты с доходом 30-50 тыс. имеют самый низкий процент наличия задолженности по кредиту. Однако, как и в случае с семейным положеним, мы видим, что эта категория представлена малым количеством людей (~1.5%), поэтому делать выводы по ней мы не можем. Аналогично, клиенты с доходом более 1 млн. и клиенты с доходом менее 30тыс. составляют менее 1% от общего количества клиентов в исходных данных, поэтому эти категории клиентов рассматривать не будем. 
Клиенты с высоким доходом - более 200 тыс., возвращают кредиты в срок чаще, чем клиенты с доходом менее 200 тыс. 

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [33]:
#построим сводную таблицу по цели кредита и рассчитаем долю клиентов с задолженностью по возврату кредита
data_pivot_category = data.pivot_table(index=['purpose_category'], values='debt', aggfunc= ['sum', 'count', lambda x: '{:.2%}'.format(x.mean())])
#переименуем столбцы
data_pivot_category = data_pivot_category.rename(columns= {'sum':'number_of_clients_with_debt', 'count':'total_number_of_clients', '<lambda>': 'percentage(%)', 'debt':''})
#отсортируем таблицу по возрастанию значений в столбце percentage(%)
data_pivot_category.sort_values(by='percentage(%)')

,number_of_clients_with_debt,total_number_of_clients,percentage(%)
,,,
purpose_category,,,
операции с недвижимостью,780,10751,7.26%
проведение свадьбы,183,2313,7.91%
получение образования,369,3988,9.25%
операции с автомобилем,400,4279,9.35%


**Вывод:** 

По итогам таблицы доля клиентов с задолженностью по возврату кредита составляет:
* Операции с недвижимостью - 7,26%
* Проведение свадьбы - 7,91%
* Получение образования - 9,25%
* Операции с автомобилем - 9,35%

Как видно из расчетов, кредиты на операции с недвижимостью и на проведение свадеб возвращают в срок чаще, чем кредиты на получение образования и на операции с автомобилем.

Учитывая то, что сумма кредитов на операции с недвижимостью в большинсте случаев превышает сумму кредитов, которые берут на автомобиль и образование, можно предположить, что банк проверяет платежеспособность клиентов с ипотекой более тщательно, чем клиентов с потребительским кредитом, что объясняет, почему клиенты, берущие кредит на недвижимость возвращают его в срок чаще клиентов с потребительским кредитом. В случае с кредитами на проведение свадьбы, можно предположить, что сумма кредита чаще всего не столь высока и поэтому клиенты чаще возвращают такие кредиты в срок. 

## 4. Общий вывод

**1. Цель исследования**

Проанализировать входные данные от банка — статистику о платёжеспособности клиентов для ответа на вопросы:
* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

**2. Ход исследования**

Данные со статистикой о клиентах мы получили из файла /datasets/data.csv. Перед анализом был проведен обзор данных, в ходе которого были выявлены и обработаны следующие ошибки:
* Выявили пропуски в столбцах days_employed и total_income и заполнили их медианными значениями по по каждому типу из столбца income_type
* Обработали аномальные значения (отрицательные значения) в столбце days_employed
* Выявили аномальные значения в столбце children и удалили строки с аномальными значениями для более корректного анализа данных
* Заменили вещественный тип данных в столбце total_income на целочисленный 
* Выявили неявные дубликаты в столбце education и удалили строки-дубликаты
* Категоризировали данные в столбце total_income по разным диапазонам дохода, а также данные в столце purpose в зависимости от целей кредита для дальнейшего анализа

**3. Итоги исследования**

Для анализа данных были применены методы группировок данных и построения свобных таблиц. По итогам анализа были сделаны следующие вывовы:

* Есть ли зависимость между количеством детей и возвратом кредита в срок?

Клиенты без детей имеют самую низкую долю задолженности по возврату кредита. У клиентов с одним ребеноком или несколькими детьми доля задолженности почти одинаковая.  

* Есть ли зависимость между семейным положением и возвратом кредита в срок?

Клиенты, состоящие в официальном браке, имеют меньше задолженностей по кредиту, по сравнению с холостыми клиентами. Доля клиентов в категории "вдовец / вдова" и у клиентов в категории "в разводе" с наличием задолженности по кредиту еще ниже, чем у клиентов состоящих в официальном браке, но так как в исходных данных клиентов в категориях "вдовец / вдова" и "в разводе" гораздо меньше, чем семейных клиентов (10% против 57%), это не позволяет нам корректно сравнивать эти две категории. 

* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Клиенты с высоким доходом - более 200 тыс., возвращают кредиты в срок чаще, чем клиенты с доходом менее 200 тыс. Клиенты с доходом 30-50 тыс. имеют самый низкий процент наличия задолженности по кредиту. Однако, как и в случае с семейным положеним, мы видим, что эта категория представлена малым количеством людей (~1.5%), поэтому делать корректные выводы по ней мы не можем. Аналогично, клиенты с доходом более 1 млн. и клиенты с доходом менее 30тыс. составляют менее 1% от общего количества клиентов в исходных данных, поэтому эти категории клиентов рассмотрены не были. 

* Как разные цели кредита влияют на его возврат в срок?

Кредиты на операции с недвижимостью и на проведение свадеб возвращают в срок чаще, чем кредиты на получение образования и на операции с автомобилем.

**Вывод:** Исходя из проведенного анализа самыми надежными клиентами для банка являются клиенты состоящие в официальном браке, не имеющие детей, с высоким доходом (более 200 тыс.) и берущие кредит на операции с недвижимостью. Тогда как клиенты не состоящие в браке, имеющие одного или нескольких детей, с доходом менее 200 тыс. и берущие кредит с целью получения образования или проведения операций с автомобилем имеют больше всего задолженностей по кредиту. 

Однако, необходимо отметить, что для более точного анализа и корректных выводов исходных данных недостаточно, а именно:
* отсутствие данных о доходе у ~10% клиентов и заполнение этих пропусков медианным значением могло повлиять на результаты расчета
* отсутстие данных о таких важных категориях, как например общая сумма кредита
* количество клиентов в некоторых категориях, как например "вдовец / вдова" или клиенты с доходом выше 1млн или ниже 50тыс. настолько мало, что не позволяет нам делать о них корректные выводы. 